In [1]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
Status: Status.OK
Toplogy: LBNL: { cpu: 6/6 , core: 186/192 , ram: 1,502/1,536 G, disk: 60,398/60,600 G, unit: 3/3 }
	Components:
		GPU-RTX6000:  GPU RTX6000 { unit: 2/2 }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 4/4 }
		NVME-P4510:  NVME P4510 { disk: 10,000/10,000 G, unit: 10/10 }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 379/381 }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2 }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 2/2 }
	Site Interfaces:
		LBNL_UKY: TrunkPort { bw: 100/100 Gbps}
		LBNL_RENC: TrunkPort { bw: 100/100 Gbps}
RENC
	Components:
	Site Interfaces:
		RENC_LBNL: TrunkPort { bw: 10/10 Gbps}
		RENC_UKY: TrunkPort { bw: 10/10 Gbps}
UKY
	Components:
	Site Interfaces:
		UKY_RENC: TrunkPort { bw: 100/100 Gbps}
		UKY_LBNL: TrunkPort { bw: 100/100 Gbps}
Links:
	port+lbnl-data-sw:HundredGigE0/0/0/0.2401-link[L2Path]: ['UKY_LBNL', 'LBNL_UKY']
	port+lbnl-data-sw:HundredGigE0

In [2]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

ssh_key_priv = None
with open (ssh_key_file_priv, "r") as myfile:
    ssh_key_priv=myfile.read()
    ssh_key_priv=ssh_key_priv.strip()

# Create Slice

In [3]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
nvme_component_type = ComponentType.NVME
nvme_model = 'P4510'
nvme_name = 'nvme1'
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='LBNL')

# Set capacities
cap = Capacities()
cap.set_fields(core=32, ram=128, disk=50)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_7')

# Add PCI devices
#n1.add_component(ctype=ComponentType.NVME, model='P4510', name='c1')

# Add node
n2 = t.add_node(name='n2', site='LBNL')

# Set properties
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_7')
# Add PCI devices
#n2.add_component(ctype=ComponentType.NVME, model='P4510', name='c1')

# Shared Cards
n1.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n1-nic1')
n2.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name='n2-nic1')

# # For Tagged Bridge, specify VLAN
# for i in t.interface_list:
#     if_labels = i.get_property(pname="labels")
#     if_labels.vlan = "200"
#     i.set_properties(labels=if_labels)

# L2Bridge Service
t.add_network_service(name='bridge1', nstype=ServiceType.L2Bridge, interfaces=[n1.interface_list[0], n2.interface_list[0]])
# L2PTP
#t.add_network_service(name='ptp', nstype=ServiceType.L2PTP, interfaces=[n1.interface_list[0], n2.interface_list[0]])

# L2STS Service
#t.add_network_service(name='sts1', nstype=ServiceType.L2STS,
#                      interfaces=[n1.interface_list[0], n2.interface_list[0]])
# Generate Slice Graph
slice_graph = t.serialize()


# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='gridftp_test', slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

Response Status Status.OK
Reservations created [{
    "graph_node_id": "2f789f0e-bad8-4691-945a-8bfb90723fde",
    "lease_end": "2021-10-22 19:15:12",
    "pending_state": "None_",
    "reservation_id": "45a9311e-17f2-4c29-b3aa-b72baf36d535",
    "reservation_state": "Unknown",
    "slice_id": "4578d603-a3a7-44fa-a02a-c3024cb932d8"
}, {
    "graph_node_id": "88d57257-52b1-4041-9612-04639f15be11",
    "lease_end": "2021-10-22 19:15:12",
    "pending_state": "None_",
    "reservation_id": "9c9a3edb-00ee-4864-99bd-79e878d3e205",
    "reservation_state": "Unknown",
    "slice_id": "4578d603-a3a7-44fa-a02a-c3024cb932d8"
}, {
    "graph_node_id": "6c0d5e47-650a-4847-b742-469d58a9a525",
    "lease_end": "2021-10-22 19:15:12",
    "pending_state": "None_",
    "reservation_id": "55aa1621-a28e-47d3-9205-9af393820cb0",
    "reservation_state": "Unknown",
    "slice_id": "4578d603-a3a7-44fa-a02a-c3024cb932d8"
}]


In [4]:

# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

In [5]:
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
else:
    print(f"Failure: {slices}")

Response Status Status.OK
Slices [{
    "graph_id": "c85f1d6b-7907-44d3-a9f8-d35973308bb0",
    "lease_end": "2021-10-22 19:15:12",
    "slice_id": "4578d603-a3a7-44fa-a02a-c3024cb932d8",
    "slice_name": "gridftp_test",
    "slice_state": "Configuring"
}]


In [8]:
status, slivers = slice_manager.slivers(slice_object=slices[0])

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))
else:
    print(f"Failure: {slivers}")

Response Status Status.OK
Slivers [{
    "allocated_capacities": "{\"core\": 32, \"disk\": 100, \"ram\": 128}",
    "allocated_labels": "{\"instance\": \"instance-000003e4\", \"instance_parent\": \"lbnl-w2.fabric-testbed.net\"}",
    "capacities": "{\"core\": 32, \"disk\": 50, \"ram\": 128}",
    "capacity_hints": "{\"instance_type\": \"fabric.c32.m128.d100\"}",
    "graph_node_id": "88d57257-52b1-4041-9612-04639f15be11",
    "join_state": "NoJoin",
    "lease_end": "2021-10-22 19:15:12",
    "management_ip": "198.129.61.46",
    "name": "n2",
    "notices": "Reservation 9c9a3edb-00ee-4864-99bd-79e878d3e205 (Slice gridftp_test(4578d603-a3a7-44fa-a02a-c3024cb932d8) Graph Id:c85f1d6b-7907-44d3-a9f8-d35973308bb0 Owner:name: orchestrator guid: orchestrator-guid email: jul@clemson.edu State:StableOK ) is in state [Active,None_]\n\nLast ticket update: \nTicket events: \nLast ticket update: \nTicket events: ",
    "pending_state": "None_",
    "reservation_id": "9c9a3edb-00ee-4864-99bd-79e878

# Get management IP of two nodes

In [9]:
server_ip = slivers[0].management_ip
server_ip

'198.129.61.46'

In [11]:
client_ip = slivers[1].management_ip
client_ip

'198.129.61.45'

# Setup gridftp on server node

In [12]:

import paramiko 
#from scp import SCPClient, SCPException
key = paramiko.RSAKey.from_private_key_file(os.environ['HOME']+"/.ssh/id_rsa")
client_server = paramiko.SSHClient()
client_server.load_system_host_keys()
client_server.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client_server.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [13]:

client_server.connect(server_ip,username='centos',pkey = key)
stdin, stdout, stderr = client_server.exec_command('sudo rpm -hUv https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Retrieving https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm
Preparing...                          ########################################
Updating / installing...
globus-toolkit-repo-6.0.9-1           ########################################
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
Loaded plugins: fastestmirror

In [14]:

stdin, stdout, stderr = client_server.exec_command('sudo yum install -y yum-plugin-priorities')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: mirror.clarkson.edu
 * extras: centos.s.uw.edu
 * updates: mirrors.usinternet.com
Resolving Dependencies
--> Running transaction check
---> Package yum-plugin-priorities.noarch 0:1.1.31-54.el7_8 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                    Arch        Version                 Repository
                                                                           Size
Installing:
 yum-plugin-priorities      noarch      1.1.31-54.el7_8         base       29 k

Transaction Summary
Install  1 Package

Total download size: 29 k
Installed size: 28 k
Public key for yum-plugin-priorities-1.1.31-54.el7_8.noarch.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : yum-plugin-priorities-1.1.31-54.el7_8.noarch                 1/1 
 

In [15]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y  wget')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.clarkson.edu
 * extras: centos.s.uw.edu
 * updates: mirrors.usinternet.com
Resolving Dependencies
--> Running transaction check
---> Package wget.x86_64 0:1.14-18.el7_6.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package        Arch             Version                   Repository      Size
Installing:
 wget           x86_64           1.14-18.el7_6.1           base           547 k

Transaction Summary
Install  1 Package

Total download size: 547 k
Installed size: 2.0 M
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : wget-1.14-18.el7_6.1.x86_64                                  1/1 
  Verifying  : wget-1.14-18.el7_6.1.x86_64                                  1/1 

Installed:
  wget.x86_64 0:1.14-18.el7_6.1                                                 

Complete!




In [16]:
stdin, stdout, stderr = client_server.exec_command('sudo wget https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))


--2021-10-21 21:54:59--  https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm
Resolving download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)... 152.19.134.145, 2610:28:3090:3001:dead:beef:cafe:fed6
Connecting to download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)|152.19.134.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97432 (95K) [application/x-rpm]
Saving to: ‘udt-4.11-6.el7.x86_64.rpm’

     0K .......... .......... .......... .......... .......... 52%  325K 0s
    50K .......... .......... .......... .......... .....     100%  311M=0.2s

2021-10-21 21:55:00 (618 KB/s) - ‘udt-4.11-6.el7.x86_64.rpm’ saved [97432/97432]




In [17]:
stdin, stdout, stderr = client_server.exec_command('sudo rpm -Uvh *rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Preparing...                          ########################################
Updating / installing...
udt-4.11-6.el7                        ########################################




In [18]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y globus-gridftp globus-gsi globus-data-management-server')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.clarkson.edu
 * extras: centos.s.uw.edu
 * updates: mirrors.usinternet.com
Resolving Dependencies
--> Running transaction check
---> Package globus-data-management-server.x86_64 0:6.0-1.el7+gt6 will be installed
--> Processing Dependency: globus-gsi-cert-utils-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-common-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-authz for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xio-pipe-driver for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xioperf for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-usage for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: gl

In [19]:
stdin, stdout, stderr = client_server.exec_command('sudo /etc/init.d/globus-gridftp-sshftp reconfigure')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Disabling sshftp access to globus-gridftp-server: OK
Successfully created /etc/gridftp-sshftp.
Enabling sshftp access to globus-gridftp-server: OK




# Setup gridftp on client node

In [20]:
client_client = paramiko.SSHClient()
client_client.load_system_host_keys()
client_client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client_client

In [21]:
client_client.connect(client_ip, username='centos',pkey = key)
stdin, stdout, stderr = client_client.exec_command('sudo rpm -hUv https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Retrieving https://downloads.globus.org/toolkit/gt6/stable/installers/repo/rpm/globus-toolkit-repo-6.0.9-1.noarch.rpm
Preparing...                          ########################################
Updating / installing...
globus-toolkit-repo-6.0.9-1           ########################################
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-stable-el7.repo to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-stable-el7.repo
Loaded plugins: fastestmirror
adding repo from: file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo
grabbing file file:///usr/share/globus/repo/globus-connect-server-5-testing-el7.repo to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
repo saved to /etc/yum.repos.d/globus-connect-server-5-testing-el7.repo
Loaded plugins: fastestmirror

In [22]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y yum-plugin-priorities')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror
Determining fastest mirrors
 * base: centos.mirrors.tds.net
 * extras: mirrors.oit.uci.edu
 * updates: mirror.sfo12.us.leaseweb.net
Resolving Dependencies
--> Running transaction check
---> Package yum-plugin-priorities.noarch 0:1.1.31-54.el7_8 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                    Arch        Version                 Repository
                                                                           Size
Installing:
 yum-plugin-priorities      noarch      1.1.31-54.el7_8         base       29 k

Transaction Summary
Install  1 Package

Total download size: 29 k
Installed size: 28 k
Public key for yum-plugin-priorities-1.1.31-54.el7_8.noarch.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : yum-plugin-priorities-1.1.31-54.el7_8.noarch          

In [23]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y  wget')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: centos.mirrors.tds.net
 * extras: mirrors.oit.uci.edu
 * updates: mirror.sfo12.us.leaseweb.net
Resolving Dependencies
--> Running transaction check
---> Package wget.x86_64 0:1.14-18.el7_6.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package        Arch             Version                   Repository      Size
Installing:
 wget           x86_64           1.14-18.el7_6.1           base           547 k

Transaction Summary
Install  1 Package

Total download size: 547 k
Installed size: 2.0 M
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : wget-1.14-18.el7_6.1.x86_64                                  1/1 
  Verifying  : wget-1.14-18.el7_6.1.x86_64                                  1/1 

Installed:
  wget.x86_64 0:1.14-18.el7_6.1                                                 

Complete!




In [24]:
stdin, stdout, stderr = client_client.exec_command('sudo wget https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))


--2021-10-21 21:55:32--  https://download-ib01.fedoraproject.org/pub/epel/7/x86_64/Packages/u/udt-4.11-6.el7.x86_64.rpm
Resolving download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)... 152.19.134.145, 2610:28:3090:3001:dead:beef:cafe:fed6
Connecting to download-ib01.fedoraproject.org (download-ib01.fedoraproject.org)|152.19.134.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97432 (95K) [application/x-rpm]
Saving to: ‘udt-4.11-6.el7.x86_64.rpm’

     0K .......... .......... .......... .......... .......... 52%  318K 0s
    50K .......... .......... .......... .......... .....     100%  480M=0.2s

2021-10-21 21:55:32 (605 KB/s) - ‘udt-4.11-6.el7.x86_64.rpm’ saved [97432/97432]




In [25]:
stdin, stdout, stderr = client_client.exec_command('sudo rpm -Uvh *rpm')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Preparing...                          ########################################
Updating / installing...
udt-4.11-6.el7                        ########################################




In [26]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y globus-gridftp globus-gsi globus-data-management-server')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: centos.mirrors.tds.net
 * extras: mirrors.oit.uci.edu
 * updates: mirror.sfo12.us.leaseweb.net
Resolving Dependencies
--> Running transaction check
---> Package globus-data-management-server.x86_64 0:6.0-1.el7+gt6 will be installed
--> Processing Dependency: globus-gsi-cert-utils-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-common-progs for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-authz for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xio-pipe-driver for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-xioperf for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing Dependency: globus-usage for package: globus-data-management-server-6.0-1.el7+gt6.x86_64
--> Processing D

In [27]:
stdin, stdout, stderr = client_client.exec_command('sudo /etc/init.d/globus-gridftp-sshftp reconfigure')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Disabling sshftp access to globus-gridftp-server: OK
Successfully created /etc/gridftp-sshftp.
Enabling sshftp access to globus-gridftp-server: OK




# setup interface on server and client nodes

In [30]:
import re
ip_of_interface_on_n1 = "192.168.10.51"
stdin, stdout, stderr = client_server.exec_command('ip a')
interface_n1 = 'eth1'
stdin, stdout, stderr = client_server.exec_command('sudo ip addr add ' + ip_of_interface_on_n1 + '/24 dev ' + interface_n1)
stdin, stdout, stderr = client_server.exec_command('sudo ip link set dev ' + interface_n1 + ' up mtu 9000')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [31]:
ip_of_interface_on_n2 = "192.168.10.52"
stdin, stdout, stderr = client_client.exec_command('ip a')
interface_n2 = 'eth1'
stdin, stdout, stderr = client_client.exec_command('sudo ip addr add ' + ip_of_interface_on_n2 + '/24 dev ' + interface_n2)
stdin, stdout, stderr = client_client.exec_command('sudo ip link set dev ' + interface_n2 + ' up mtu 9000')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

# TCP buffing size tuning on server and client node

In [32]:
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.rmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.wmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_rmem = 4096 87380 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_wmem = 4096 65536 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_congestion_control=htcp" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.ipv4.tcp_mtu_probing=1" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_server.exec_command( 'echo "net.core.default_qdisc = fq" | sudo tee -a /etc/sysctl.conf')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.default_qdisc = fq

stderr 


In [33]:
stdin, stdout, stderr = client_server.exec_command('sudo sysctl -p')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.rmem_max = 2147482548
net.core.wmem_max = 2147482548
net.ipv4.tcp_rmem = 4096 87380 1073741274
net.ipv4.tcp_wmem = 4096 65536 1073741274
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq

stderr 


In [34]:

stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.rmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.wmem_max = 2147482548" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_rmem = 4096 87380 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_wmem = 4096 65536 1073741274" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_congestion_control=htcp" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.ipv4.tcp_mtu_probing=1" | sudo tee -a /etc/sysctl.conf')
stdin, stdout, stderr = client_client.exec_command( 'echo "net.core.default_qdisc = fq" | sudo tee -a /etc/sysctl.conf')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.default_qdisc = fq

stderr 


In [35]:
stdin, stdout, stderr = client_client.exec_command('sudo sysctl -p')
print("stdout", stdout.read().decode("utf-8"))
print("stderr", stderr.read().decode("utf-8"))

stdout net.core.rmem_max = 2147482548
net.core.wmem_max = 2147482548
net.ipv4.tcp_rmem = 4096 87380 1073741274
net.ipv4.tcp_wmem = 4096 65536 1073741274
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq

stderr 


# iperf test

In [36]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y epel-release')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.clarkson.edu
 * extras: centos.s.uw.edu
 * updates: mirrors.usinternet.com
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                Arch             Version         Repository        Size
Installing:
 epel-release           noarch           7-11            extras            15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11                                                    

Complete!




In [37]:
stdin, stdout, stderr = client_server.exec_command('sudo yum install -y iperf')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: mirror.clarkson.edu
 * epel: mirrors.wcupa.edu
 * extras: centos.s.uw.edu
 * updates: mirrors.usinternet.com
155 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package iperf.x86_64 0:2.0.13-1.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package         Arch             Version                  Repository      Size
Installing:
 iperf           x86_64           2.0.13-1.el7             epel            86 k

Transaction Summary
Install  1 Package

Total download size: 86 k
Installed size: 185 k
Public key for iperf-2.0.13-1.el7.x86_64.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : iperf-2.0.13-1.el7.x86_64                                    1/1 


In [38]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y epel-release')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: centos.mirrors.tds.net
 * extras: mirrors.oit.uci.edu
 * updates: mirror.sfo12.us.leaseweb.net
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                Arch             Version         Repository        Size
Installing:
 epel-release           noarch           7-11            extras            15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11                                                    

Complete!




In [39]:
stdin, stdout, stderr = client_client.exec_command('sudo yum install -y iperf')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Loaded plugins: fastestmirror, priorities
Loading mirror speeds from cached hostfile
 * base: centos.mirrors.tds.net
 * epel: mirror.prgmr.com
 * extras: mirrors.oit.uci.edu
 * updates: mirror.sfo12.us.leaseweb.net
155 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package iperf.x86_64 0:2.0.13-1.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package         Arch             Version                  Repository      Size
Installing:
 iperf           x86_64           2.0.13-1.el7             epel            86 k

Transaction Summary
Install  1 Package

Total download size: 86 k
Installed size: 185 k
Public key for iperf-2.0.13-1.el7.x86_64.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : iperf-2.0.13-1.el7.x86_64                             

In [50]:
stdin, stdout, stderr = client_server.exec_command('iperf -s -P32 -w 32M > /dev/null 2>&1 &')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [51]:
stdin, stdout, stderr = client_client.exec_command('iperf -c ' + ip_of_interface_on_n1 + ' -P 32 -w 32M')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

------------------------------------------------------------
Client connecting to 192.168.10.51, TCP port 5001
TCP window size: 64.0 MByte (WARNING: requested 32.0 MByte)
------------------------------------------------------------
[ 32] local 192.168.10.52 port 55782 connected with 192.168.10.51 port 5001
[  3] local 192.168.10.52 port 55720 connected with 192.168.10.51 port 5001
[  4] local 192.168.10.52 port 55722 connected with 192.168.10.51 port 5001
[  6] local 192.168.10.52 port 55726 connected with 192.168.10.51 port 5001
[  5] local 192.168.10.52 port 55724 connected with 192.168.10.51 port 5001
[ 10] local 192.168.10.52 port 55728 connected with 192.168.10.51 port 5001
[  8] local 192.168.10.52 port 55730 connected with 192.168.10.51 port 5001
[ 11] local 192.168.10.52 port 55732 connected with 192.168.10.51 port 5001
[  7] local 192.168.10.52 port 55736 connected with 192.168.10.51 port 5001
[ 14] local 192.168.10.52 port 55734 connected with 192.168.10.51 port 5001
[  9] lo

# gridftp performance test

In [52]:
stdin, stdout, stderr = client_server.exec_command('sudo globus-gridftp-server -control-interface 0.0.0.0  -aa  -p 8080 -anonymous-user 1040 -home-dir /tmp/ -auth-level 0 -allow-root
 > /dev/null 2>&1 &')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

SyntaxError: EOL while scanning string literal (<ipython-input-52-2f0f0a0f32db>, line 1)

In [57]:
stdin, stdout, stderr = client_server.exec_command('sudo fallocate -l 80G gentoo_root.img')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [58]:
stdin, stdout, stderr = client_client.exec_command('sudo globus-url-copy -vb -p 32 ftp://192.168.10.51:4000/home/centos/gentoo_root.img file:///dev/null')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Source: ftp://192.168.10.51:4000/dev/
Dest:   file:///dev/
  zero  ->  null

            0 bytes         0.00 MB/sec avg         0.00 MB/sec inst


error: globus_xio: Unable to connect to 192.168.10.51:4000
globus_xio: System error in connect: No route to host
globus_xio: A system call failed: No route to host



# release recources

In [59]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

gridftp_test:
   ID         : 460a17ff-8aeb-4285-9b9e-553e72261682
   State      : StableOK
   Lease End  : 2021-10-20 01:59:48



In [60]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))

Response Status Status.OK
Response received None
